In [1]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [2]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [3]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [4]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [5]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [200]>

In [7]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [8]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [9]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2091\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1781\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 628\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 179"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [15]:
def grades_data(grades):
    data = {}
    
    for grade in grades:
        example_naep_query = (
            'https://www.nationsreportcard.gov/'
            'Dataservice/GetAdhocData.aspx?'
            'type=data&subject=writing&grade&'
            'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
        response = requests.get(example_naep_query)
        
        try:
            grade_data = response.json()
            data[grade] = grade_data
        except Exception as e:
            print('Failed to get result from API due to error:')
            print(e)
    return data

grades = [4, 8, 12]
grades_data(grades)

## I needed to get the right URL - look at solutions

Failed to get result from API due to error:
Invalid control character at: line 1 column 110 (char 109)
Failed to get result from API due to error:
Invalid control character at: line 1 column 110 (char 109)
Failed to get result from API due to error:
Invalid control character at: line 1 column 110 (char 109)


{}

# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [ ]:
## get the key
API_KEY = "PASTE KEY HERE"

In [ ]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


In [ ]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

In [ ]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [25]:
API_KEY = "1Gh9eKod5_-vX1otu_KEzcFphmJC58Kwb2qsJbcKPn76WUI7eW6biYfFe6R6Pmx_7NRaaQraF-Co_eLwcjvLl-DGQ4A0wHE8Z6WlFm9Bum6lEFvh9wiLufavouMtZXYx"

# new location

base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Pitsea,Essex,SS13 2AJ"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()



<Response [200]>

In [26]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'vmYD90yeG5dHj5UmppKjqQ',
 'alias': 'domino-s-basildon',
 'name': 'Domino’s',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/w6zSi7feoTX-ed_61qxSgg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/domino-s-basildon?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=AYAiHNSGxz_RRHzq3cO46w',
 'review_count': 1,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
 'rating': 1.0,
 'coordinates': {'latitude': 51.5645332648766, 'longitude': 0.504321381675753},
 'transactions': [],
 'location': {'address1': '26-28 Station Lane',
  'address2': '',
  'address3': None,
  'city': 'Basildon',
  'zip_code': 'SS13 3BG',
  'country': 'GB',
  'state': 'ESS',
  'display_address': ['26-28 Station Lane',
   'Basildon SS13 3BG',
   'United Kingdom']},
 'phone': '+441268556555',
 'display_phone': '+44 1268 556555',
 'distance': 1020.8399986553987}

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,vmYD90yeG5dHj5UmppKjqQ,domino-s-basildon,Domino’s,https://s3-media2.fl.yelpcdn.com/bphoto/w6zSi7...,False,https://www.yelp.com/biz/domino-s-basildon?adj...,1,"[{'alias': 'pizza', 'title': 'Pizza'}]",1.0,"{'latitude': 51.5645332648766, 'longitude': 0....",[],"{'address1': '26-28 Station Lane', 'address2':...",+441268556555,+44 1268 556555,1020.839999,NaN
1,TytCwDOTkTGbhzwbgKcNZA,beefeater-grill-basildon,Beefeater Grill,,False,https://www.yelp.com/biz/beefeater-grill-basil...,1,"[{'alias': 'steak', 'title': 'Steakhouses'}]",2.0,"{'latitude': 51.5858076, 'longitude': 0.498689}",[],"{'address1': 'East Mayne Basildon', 'address2'...",+441268522227,+44 1268 522227,1634.202611,NaN
2,ba0QR9llW5Ux-xA6Lu4ngw,mintoos-tandoori-basildon,Mintoos Tandoori,,False,https://www.yelp.com/biz/mintoos-tandoori-basi...,1,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",5.0,"{'latitude': 51.5822124, 'longitude': 0.4876271}",[],"{'address1': '94 Whitmore Way', 'address2': No...",+441268284193,+44 1268 284193,1867.198027,££
3,K-wdOhqtfSLZiLU7S6zNHQ,neds-noodle-bar-basildon,ned's noodle bar,,False,https://www.yelp.com/biz/neds-noodle-bar-basil...,2,"[{'alias': 'restaurants', 'title': 'Restaurant...",3.0,"{'latitude': 51.5730896, 'longitude': 0.46428}",[],"{'address1': 'Unit 3 The Food Terrace', 'addre...",+441268522228,+44 1268 522228,1521.905203,NaN
4,VdPM-3S1rF1ebCoXkgrX-Q,dil-tandoori-basildon,Dil Tandoori,,False,https://www.yelp.com/biz/dil-tandoori-basildon...,1,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",3.0,"{'latitude': 51.5651516, 'longitude': 0.5071876}",[],"{'address1': '79 High Road', 'address2': None,...",+441268584858,+44 1268 584858,892.663110,NaN


In [24]:
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,phone,display_phone,price
vmYD90yeG5dHj5UmppKjqQ,vmYD90yeG5dHj5UmppKjqQ,domino-s-basildon,Domino’s,https://s3-media2.fl.yelpcdn.com/bphoto/w6zSi7...,https://www.yelp.com/biz/domino-s-basildon?adj...,+441268556555,+44 1268 556555,NaN
TytCwDOTkTGbhzwbgKcNZA,TytCwDOTkTGbhzwbgKcNZA,beefeater-grill-basildon,Beefeater Grill,,https://www.yelp.com/biz/beefeater-grill-basil...,+441268522227,+44 1268 522227,NaN
ba0QR9llW5Ux-xA6Lu4ngw,ba0QR9llW5Ux-xA6Lu4ngw,mintoos-tandoori-basildon,Mintoos Tandoori,,https://www.yelp.com/biz/mintoos-tandoori-basi...,+441268284193,+44 1268 284193,££
K-wdOhqtfSLZiLU7S6zNHQ,K-wdOhqtfSLZiLU7S6zNHQ,neds-noodle-bar-basildon,ned's noodle bar,,https://www.yelp.com/biz/neds-noodle-bar-basil...,+441268522228,+44 1268 522228,NaN
VdPM-3S1rF1ebCoXkgrX-Q,VdPM-3S1rF1ebCoXkgrX-Q,dil-tandoori-basildon,Dil Tandoori,,https://www.yelp.com/biz/dil-tandoori-basildon...,+441268584858,+44 1268 584858,NaN
cJxq-FSAESemcRpFeJj8zg,cJxq-FSAESemcRpFeJj8zg,subway-basildon,Subway,,https://www.yelp.com/biz/subway-basildon?adjus...,+441268557755,+44 1268 557755,NaN
4cV2mWv4m9juL_2Lmi2rcQ,4cV2mWv4m9juL_2Lmi2rcQ,asia-spice-basildon,asia spice,,https://www.yelp.com/biz/asia-spice-basildon?a...,,,NaN


In [30]:
import requests

url = "https://api.yelp.com/v3/businesses/cJxq-FSAESemcRpFeJj8zg/reviews?limit=20&sort_by=yelp_sort"

headers = {"accept": "application/json"}

response = requests.get(url, headers= header)

print(response.text)

{"reviews": [{"id": "xDZONSOhpHD5lgU9xuLIsg", "url": "https://www.yelp.com/biz/subway-basildon?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=xDZONSOhpHD5lgU9xuLIsg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w", "text": "Not bad it's a subway so you know what your getting. Issue with this particular one is first visit out of meatballs second visit out of turkey third visit...", "rating": 3, "time_created": "2017-10-24 11:59:59", "user": {"id": "A3nTfTjTcKLw1BkuUlYSCA", "profile_url": "https://www.yelp.com/user_details?userid=A3nTfTjTcKLw1BkuUlYSCA", "image_url": null, "name": "Timothy T."}}, {"id": "0pZ4y0l-t1_1xw-buaX1yg", "url": "https://www.yelp.com/biz/subway-basildon?adjust_creative=AYAiHNSGxz_RRHzq3cO46w&hrid=0pZ4y0l-t1_1xw-buaX1yg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=AYAiHNSGxz_RRHzq3cO46w", "text": "This Subway has recently re opened. The new staff are very friendly and helpful.\n\nIt is clean and t